# How to fine-tune a pre-trained model for the task of paraphrase identification WITHOUT USING TRAINER 
(Pytorch version)

In the previous notebook, we will use a Trainer to fine-tune a pre-trained model (bert-base_uncased) for the task of paraphrase identification. 

Now, we will try to replicate the Trainer by hand.

In [1]:
!pip install datasets
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 29.7 MB/s 
     |████████████████████████████████| 86 kB 7.1 MB/s 
     |████████████████████████████████| 140 kB 74.4 MB/s 
     |████████████████████████████████| 1.1 MB 59.4 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 212 kB 59.6 MB/s 
     |████████████████████████████████| 596 kB 55.1 MB/s 
     |████████████████████████████████| 127 kB 30.4 MB/s 
     |████████████████████████████████| 144 kB 57.4 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 75.6 MB/s 
     |████████████████████████████████| 112 kB 72.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing in

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

## Preparing the training

We will need to define a few objects. 

- dataloaders, which are used to iterate over batches. But before we can define those dataloaders, we need to apply a bit of postprocessing to our tokenized_datasets. Specifically, we need to:

- Remove the columns corresponding to values the model does not expect (like the sentence1 and sentence2 columns).
- Rename the column label to labels (because the model expects the argument to be named labels).
- Set the format of the datasets so they return PyTorch tensors instead of lists.

Our tokenized_datasets has one method for each of those steps:



In [3]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names # the model will only accept these fields

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

We can now define the dataloaders objects:

In [4]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

Let'us to check some batches. Every batch always has 8 instances. However, the maximum length of sequences may vary from batch to batch. 

In [5]:
i = 0
for batch in train_dataloader:
    if i == 3:
        break
    i += 1
    print({k: v.shape for k, v in batch.items()})



{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 74]), 'token_type_ids': torch.Size([8, 74]), 'attention_mask': torch.Size([8, 74])}
{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 64]), 'token_type_ids': torch.Size([8, 64]), 'attention_mask': torch.Size([8, 64])}
{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 71]), 'token_type_ids': torch.Size([8, 71]), 'attention_mask': torch.Size([8, 71])}


We create the model:

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Before fine-tuning the model on the entire dataset, let's do a little test on a single batch to check if it is fine.



In [7]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape) # All models will return the loss when labels are provided, 
                                            # we also get the logits (two for each input in our batch, so a tensor of size 8 x 2).




tensor(0.6252, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


To replicate the Trainer, we also need to define:
- an optimizer: in our case, we will use AdamW, the same optimizer used by Trainer. 
- a learning rate scheduler: we will use: linear decay from the maximum value (5e-5) to 0. 


In [8]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Moreover, we should define the number of epochs. As the Trainer uses three epochs by default, we will also use three.

The number of training steps for our model can be calculated as the number of epochs multiplied by the number of training batches (the size of the training loader):

In [9]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps) # num_epochs * lenght of train_dataloader
print(num_epochs*len(train_dataloader))

1377
1377


We must check that we are using GPU:

In [10]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

## Training loop 
Finally, we are ready to train our model:

In [11]:
from tqdm.auto import tqdm  # adds progress bar over our number of training steps

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()     # Back Propagation: it does is compute gradient of loss 



        optimizer.step()    # update your model parameters: MOdify the parameters by subtracting the gradient
        lr_scheduler.step() # lr_scheduler provides several methods to adjust the learning rate based on the number of epochs; dynamic learning rate
        optimizer.zero_grad()   # clear the gradients from the previous training step
        progress_bar.update(1)


  0%|          | 0/1377 [00:00<?, ?it/s]

## Evaluation loop

The previous loop does not provide any information about the metrics. So we need to implement an evaluation loop, which will calculate the metrics for each batch.
Moreover, the object metric can accumulate batchs (with the add_batch), so we will calculate the final results with the function metric.compuct():


In [12]:
from datasets import load_metric

metric = load_metric("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():               # this disables gradient calculation.
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.6838235294117647, 'f1': 0.8122270742358079}

## Accelerate library (for lucky people)

There is a library, Accelerate library, allows us to enable distributed training on multiple GPUs or TPUs. 
Unfortunately, we do not have access to multiple GPUs or TPUs.

https://github.com/huggingface/accelerate

